# Stage 2: Content Prioritization - Unit Tests

This notebook tests the **Content Prioritization** service in isolation.

**What it tests:**
- Temporal priority calculation (earnings proximity)
- Source prioritization based on GRID config
- Subquery generation
- Priority reasoning

In [ ]:
# Setup path
import sys
sys.path.insert(0, "../..")

from cmpt.services import (
    ContentPrioritizationService,
    ContextBuilderOutput,
    CompanyInfo,
    TemporalContext,
    Priority,
    DataSource,
)

## Helper: Create Mock Context Output

In [ ]:
def create_mock_context(
    company_name: str = "Apple Inc",
    ticker: str = "AAPL",
    meeting_date: str = "2025-02-15",
    earnings_date: str | None = None,
    days_to_earnings: int | None = None,
    market_cap: str = "Large Cap",
) -> ContextBuilderOutput:
    """Create a mock ContextBuilderOutput for testing."""
    return ContextBuilderOutput(
        company_info=CompanyInfo(
            name=company_name,
            ticker=ticker,
            industry="Technology",
            sector="Information Technology",
            market_cap=market_cap,
        ),
        company_name=company_name,
        ticker=ticker,
        temporal_context=TemporalContext(
            meeting_date=meeting_date,
            event_dt=earnings_date,
            fiscal_quarter="1",
            fiscal_year="2025",
            days_to_earnings=days_to_earnings,
            news_lookback_days=30,
            filing_quarters=8,
        ),
    )

print("✓ Helper function created")

## Test 1: Near Earnings Priority (Earnings Dominant)

In [ ]:
async def test_near_earnings_priority():
    """Test that near-earnings meetings prioritize earnings data."""
    service = ContentPrioritizationService()
    
    # Meeting 3 days before earnings
    context = create_mock_context(
        meeting_date="2025-02-15",
        earnings_date="2025-02-18",
        days_to_earnings=3,
    )
    
    output = await service.execute(context)
    
    # Verify earnings-dominant priority
    assert output.priority_distribution is not None
    
    # Earnings should have highest weight
    earnings_weight = output.priority_distribution.get("earnings_agent", 0)
    news_weight = output.priority_distribution.get("news_agent", 0)
    
    print(f"✓ Priority Distribution (Near Earnings):")
    for agent, weight in output.priority_distribution.items():
        print(f"    {agent}: {weight}%")
    
    assert earnings_weight > news_weight, "Earnings should be prioritized near earnings"
    print(f"✓ Earnings prioritized over news ({earnings_weight}% > {news_weight}%)")
    return output

output = await test_near_earnings_priority()

## Test 2: Far from Earnings Priority (News Dominant)

In [ ]:
async def test_far_from_earnings_priority():
    """Test that meetings far from earnings prioritize news."""
    service = ContentPrioritizationService()
    
    # Meeting 60 days before earnings
    context = create_mock_context(
        meeting_date="2025-01-15",
        earnings_date="2025-03-15",
        days_to_earnings=60,
    )
    
    output = await service.execute(context)
    
    # News should have higher weight
    earnings_weight = output.priority_distribution.get("earnings_agent", 0)
    news_weight = output.priority_distribution.get("news_agent", 0)
    
    print(f"✓ Priority Distribution (Far from Earnings):")
    for agent, weight in output.priority_distribution.items():
        print(f"    {agent}: {weight}%")
    
    assert news_weight > earnings_weight, "News should be prioritized far from earnings"
    print(f"✓ News prioritized over earnings ({news_weight}% > {earnings_weight}%)")
    return output

output = await test_far_from_earnings_priority()

## Test 3: Subquery Generation

In [ ]:
async def test_subquery_generation():
    """Test that subqueries are generated for each agent."""
    service = ContentPrioritizationService()
    
    context = create_mock_context(ticker="MSFT")
    output = await service.execute(context)
    
    # Verify subqueries
    assert len(output.subqueries) > 0
    print(f"✓ Generated {len(output.subqueries)} subqueries")
    
    # Check each subquery has required fields
    for sq in output.subqueries:
        assert sq.agent is not None
        assert sq.query is not None
        assert sq.params is not None
        print(f"    - {sq.agent}: {sq.query[:30]}... (params: {list(sq.params.keys())})")
    
    # Verify subqueries_by_agent grouping
    assert len(output.subqueries_by_agent) > 0
    print(f"\n✓ Subqueries grouped by {len(output.subqueries_by_agent)} agents")
    return output

output = await test_subquery_generation()

## Test 4: Subquery Parameters from Temporal Context

In [ ]:
async def test_subquery_params():
    """Test that subquery params include temporal context."""
    service = ContentPrioritizationService()
    
    context = create_mock_context(
        ticker="GOOGL",
        meeting_date="2025-04-15",
    )
    # Update temporal context
    context.temporal_context.fiscal_year = "2025"
    context.temporal_context.fiscal_quarter = "2"
    context.temporal_context.news_lookback_days = 45
    
    output = await service.execute(context)
    
    # Check earnings subquery has fiscal params
    earnings_sq = next((sq for sq in output.subqueries if "earnings" in sq.agent.lower()), None)
    if earnings_sq:
        assert earnings_sq.params.get("ticker") == "GOOGL"
        assert earnings_sq.params.get("fiscal_year") == "2025"
        assert earnings_sq.params.get("fiscal_quarter") == "Q2"
        print(f"✓ Earnings subquery params:")
        for k, v in earnings_sq.params.items():
            print(f"    {k}: {v}")
    
    # Check news subquery has lookback
    news_sq = next((sq for sq in output.subqueries if "news" in sq.agent.lower()), None)
    if news_sq:
        assert news_sq.params.get("days") == 45
        print(f"\n✓ News subquery params:")
        for k, v in news_sq.params.items():
            print(f"    {k}: {v}")
    
    return output

output = await test_subquery_params()

## Test 5: Source Prioritization

In [ ]:
async def test_source_prioritization():
    """Test that sources are correctly prioritized."""
    service = ContentPrioritizationService()
    
    context = create_mock_context(days_to_earnings=5)  # Near earnings
    output = await service.execute(context)
    
    # Get primary sources
    primary = [s for s in output.prioritized_sources if s.priority == Priority.PRIMARY]
    secondary = [s for s in output.prioritized_sources if s.priority == Priority.SECONDARY]
    
    print(f"✓ Prioritized Sources:")
    print(f"  Primary ({len(primary)}):")
    for s in primary:
        print(f"    - {s.source.value}")
    print(f"  Secondary ({len(secondary)}):")
    for s in secondary:
        print(f"    - {s.source.value}")
    
    # Near earnings: SEC and Earnings should be primary
    primary_sources = [s.source for s in primary]
    assert DataSource.SEC_FILING in primary_sources or DataSource.EARNINGS in primary_sources
    print(f"\n✓ Correct sources prioritized for near-earnings meeting")
    return output

output = await test_source_prioritization()

## Test 6: Prioritization Reasoning

In [ ]:
async def test_prioritization_reasoning():
    """Test that reasoning is generated."""
    service = ContentPrioritizationService()
    
    context = create_mock_context(
        company_name="Tesla Inc",
        days_to_earnings=10,
        market_cap="Large Cap",
    )
    
    output = await service.execute(context)
    
    # Verify reasoning exists
    assert output.prioritization_reasoning is not None
    assert len(output.prioritization_reasoning) > 0
    
    print(f"✓ Prioritization Reasoning:")
    print(output.prioritization_reasoning)
    
    # Should mention company name
    assert "Tesla" in output.prioritization_reasoning
    return output

output = await test_prioritization_reasoning()

## Test 7: Custom GRID Config

In [ ]:
async def test_custom_grid_config():
    """Test with custom GRID configuration."""
    custom_config = {
        "earnings_proximity_weeks": 2,  # Extended window
        "news_lookback_days": {"large_cap": 60, "mid_cap": 90, "small_cap": 120},
        "filing_quarters": {"revenue": 12, "other": 6},
        "max_news_results": 100,
        "max_filing_results": 30,
        "include_filing_types": ["10-K", "10-Q", "8-K", "S-1"],
    }
    
    service = ContentPrioritizationService(grid_config=custom_config)
    context = create_mock_context(days_to_earnings=10)  # Within 2 weeks
    
    output = await service.execute(context)
    
    # Verify custom config was used
    assert output.grid_config == custom_config
    
    # With 2-week window, 10 days should still be "near earnings"
    primary = [s for s in output.prioritized_sources if s.priority == Priority.PRIMARY]
    
    print(f"✓ Custom GRID config applied")
    print(f"✓ Earnings proximity window: {custom_config['earnings_proximity_weeks']} weeks")
    print(f"✓ Max news results: {custom_config['max_news_results']}")
    return output

output = await test_custom_grid_config()

## Summary

In [ ]:
print("="*60)
print("  Stage 2: Content Prioritization - All Tests Passed!")
print("="*60)
print()
print("Tested:")
print("  ✓ Near earnings priority (earnings dominant)")
print("  ✓ Far from earnings priority (news dominant)")
print("  ✓ Subquery generation")
print("  ✓ Subquery parameters from temporal context")
print("  ✓ Source prioritization")
print("  ✓ Prioritization reasoning")
print("  ✓ Custom GRID configuration")